In [1]:
import os


# Read in poker hands as a list then split into 2
poker_file = os.getcwd()+"\\poker.txt"
with open(poker_file, "r") as pf:
    all_hands = (pf
                 .read()
                 .split("\n"))

In [2]:
# Divide cards dealt into the respective hands
player1 = [h[0:14].split() for h in all_hands]
player2 = [h[15:].split() for h in all_hands]

In [3]:
from collections import Counter

def hand_cnt(cards:list)->(int,dict):
    card_nums = [c[0] for c in cards]
    cnt_dict = Counter(card_nums)
    max_dup_cnt = max(v for k,v in cnt_dict.items())
    return max_dup_cnt, cnt_dict

def card_dict()->dict:
    """
    Dictionary to map card strings values between 2 to 14
    """
    val_dict = {"2":2,
                "3":3,
                "4":4,
                "5":5,
                "6":6,
                "7":7,
                "8":8,
                "9":9,
                "T":10,
                "J":11,
                "Q":12,
                "K":13,
                "A":14}
    return val_dict

In [4]:
def check_straight(count_dict:dict)->bool:
    " Check if hand contains a straight"
    # Sort cards by value from 2 to 14
    card_nums = [k for k,v in count_dict.items()]
    card_value = card_dict()
    hand_values = [card_value[num] for num in card_nums]
    hand_values.sort()
    # Possible straight conditions
    if (hand_values[-1]-hand_values[0])==4:
        straight_status = True
    elif hand_values == [2,3,4,5,14]:
        straight_status = True
    else:
        straight_status = False
    return straight_status

In [5]:
def get_rank(hand:list, cnt:int, cnt_dict:dict)->int:
    """
    Determine the general rank, with 1 being the least,
        of hand dealt using the following scheme:
    High card -> 1
    Pair -> 2
    Two pair -> 3
    Three of a kind -> 4
    Straight -> 5
    Flush -> 6
    Full House -> 7
    Four of a kind -> 8
    Straight Flush -> 9
    Royal Flush -> 10
    """
    if cnt == 1:
        # Check for straight
        straight = check_straight(cnt_dict)
        # Check for Flush
        suits = [h[1] for h in hand]
        same_suit = all([su==suits[0] for su in suits])
        if same_suit:
            # Check for Straight Flush
            if straight:
                # Royal Flush Condition
                cnt_key = [k for k, v in cnt_dict.items()]
                if ("T" in cnt_key) and ("A" in cnt_key):
                    rank = 10
                else:
                    rank = 9
            else:
                rank = 6
        # Check for Straight
        elif straight:
            rank = 5
        else:
            rank = 1
    elif cnt == 2:
        cnt_values = [v for k, v in cnt_dict.items()]
        cnt_values.remove(2)
        if 2 in cnt_values:
            rank = 3
        else:
            rank = 2
    elif cnt == 3:
        cnt_values = [v for k, v in cnt_dict.items()]
        if (2 in cnt_values):
            rank = 7
        else:
            rank = 4
    elif cnt == 4:
        rank = 8
    return rank

In [6]:
def get_hand_ranks(player_hands:list)->list:
    """
    Takes a list of hands (i.e. list of list) and ranks each sublist.
    """
    all_ranks = []
    for hand in player_hands:
        max_dup, count_dictoinary = hand_cnt(hand)
        hand_rank = get_rank(hand, max_dup, count_dictoinary)
        all_ranks.append(hand_rank)
    return all_ranks

In [7]:
# Rank each hand
p1_ranks = get_hand_ranks(player1)
p2_ranks = get_hand_ranks(player2)

In [8]:
def hand_cnt_val(cards:list)->(dict):
    """
    Like hand_cnt() function but returns result using car values instead.
    e.g. A-> 14
    """
    card_value = card_dict()
    card_nums = [card_value[c[0][0]] for c in cards]
    cnt_dict = Counter(card_nums)
    max_dup_cnt = max(v for k,v in cnt_dict.items())
    return max_dup_cnt, cnt_dict

In [9]:
def compare_max(listA:list,listB:list)->int:
    """
    Compare to see which of the two lists has the maximum element.
    If A, return 1. If B, return 0. Else, return -10**6
    """
    ind = -10**6 # Tie
    while len(listA) > 0:
        if max(listA) > max(listB):
            ind = 1
            return ind
        elif max(listB) > max(listA):
            ind = 0
            return ind
        else:
            listA.remove(max(listA))
            listB.remove(max(listB))
    return ind

In [10]:
def tie_breaker(handA:list, handB:list)->int:
    """
    Iterate to determine the winning hand.
    """
    max_cnt_A, cnt_of_A = hand_cnt_val(handA)
    main_A = [k for k,v in cnt_of_A .items() if v==max_cnt_A]
    max_cnt_B, cnt_of_B = hand_cnt_val(handB)
    main_B = [k for k,v in cnt_of_B .items() if v==max_cnt_B]
    main_ind = compare_max(main_A,main_B)
    if main_ind >=0:
        final_ind = main_ind
    else: # Tie
        s_A = [k for k,v in cnt_of_A .items() if v!=max_cnt_A]
        s_B = [k for k,v in cnt_of_B .items() if v!=max_cnt_B]
        s_ind = compare_max(s_A, s_B)
        final_ind = s_ind
    return final_ind

In [11]:
def compare_hands(setA_ranking:list,
                  setB_ranking:list,
                  handA:list,
                  handB:list)->list:
    """
    Comparing hands while assuming that there is always a clear winner.
    A wins then 1.
    B wins then 0.
    """
    # Check to make sure inputs are valid
    if len(setA_ranking)!=len(setB_ranking):
        print("Not the same number of hands")
        return
    # Compare hands
    winner = []
    for i in range(0, len(handA)):
        if setA_ranking[i] > setB_ranking[i]:
            winner.append(1)
        elif setA_ranking[i] < setB_ranking[i]:
            winner.append(0)
        else: # Tie
            # Compare winning hand max value
            if setA_ranking[i]==10:
                print("Error, a true tie has occured")
                winner.append(-10**6) #Error, true tie has occured
            else:
                # Tie then remove and iterate
                winning_ind = tie_breaker(handA[i],handB[i])
                winner.append(winning_ind)
    return winner

In [12]:
# Determine player 1 wins
final_result = compare_hands(p1_ranks,
                             p2_ranks,
                             player1,
                             player2)
print(f"Player1 won {sum(final_result)} out of {len(player1)} hands")

Player1 won 376 out of 1000 hands
